In [22]:
import pandas as pd
import re
from transformers import AutoTokenizer

tokenizer_name = "dbmdz/bert-base-turkish-uncased"
LOWERCASE = True

try:
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    print(f"BERTurk tokenizer ({tokenizer_name}) başarıyla yüklendi.")
except OSError:
    print(f"HATA: BERTurk tokenizer ({tokenizer_name}) bulunamadı veya indirilemedi.")
    print("İnternet bağlantınızı kontrol edin veya model adının doğru olduğundan emin olun.")
    raise # Hata oluştuysa devam etme

BERTurk tokenizer (dbmdz/bert-base-turkish-uncased) başarıyla yüklendi.


In [3]:
df = pd.read_csv("/content/turkish_movie_sentiment_dataset.csv")

In [4]:
df.head()

,comment,film_name,point
0,\n Jean Reno denince zate...,Sevginin Gücü,"5,0"
1,\n Ekşın falan izlemek is...,Sevginin Gücü,"5,0"
2,\n Bu yapım hakkında öyle...,Sevginin Gücü,"5,0"
3,\n finali yeter... (sting...,Sevginin Gücü,"5,0"
4,\n Jean Reno..\nbu adam k...,Sevginin Gücü,"5,0"


In [5]:
df.drop(columns=["point"], inplace=True)
df["comment"]=df["comment"].str.replace('\n', "")
df['comment'] = df['comment'].str.strip()
df['comment'] = df['comment'].str.replace('"', '')

In [6]:
df.head()

,comment,film_name
0,Jean Reno denince zaten leon filmi gelir akla ...,Sevginin Gücü
1,Ekşın falan izlemek istiyorsanız eğer bunu izl...,Sevginin Gücü
2,Bu yapım hakkında öyle çok şey yazabilirim ki ...,Sevginin Gücü
3,finali yeter... (sting - shape of my heart)baz...,Sevginin Gücü
4,Jean Reno..bu adam kusursuz biri..ve oyunculug...,Sevginin Gücü


In [15]:
chars_to_normalize_map_from = "ÂâÎîÛûÊêÄäËëÏïÖöÜü"
chars_to_normalize_map_to   = "AaIiUuEeAaEeIiOoUu"

translation_table = str.maketrans(chars_to_normalize_map_from, chars_to_normalize_map_to)

def preprocess_for_berturk(text):
  """
  Türkçe metinleri BERTurk modeline uygun hale getirmek için temizleyceğiz
  """
  if not isinstance(text, str) or not text.strip():
    return ""

  if LOWERCASE:
    text = text.lower()

  # ---- Karakter Normalizasyonu ---
  text = text.translate(translation_table)

  # ---- URL'lerin kaldırılması ---
  text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

  # ---- @mention'ların kaldırılması ---
  text = re.sub(r'\@\w+', '', text)

  # ---- hashtag isaretinin kaldrılması ---
  text = text.replace("#", "")

  text = re.sub(r'[^a-z0-9ığüşöç\s.,!?:;]', '', text)

  text = re.sub(r'[.,!?:;\(\)\[\]\{\}]', '', text)

  # ---- Ekstra boşlukları kaldırma ---
  text = re.sub(r'\s+', ' ', text).strip()

  return text


print("Karakter normalizasyonu içeren ön işleme fonksiyonu 'preprocess_for_berturk_normalized' tanımlandı.")

Karakter normalizasyonu içeren ön işleme fonksiyonu 'preprocess_for_berturk_normalized' tanımlandı.


In [16]:
print("\nBERTurk için ön işleme (normalizasyonlu) BAŞLIYOR...")
if 'comment' not in df.columns:
  raise ValueError("DataFrame'de 'comment' sütunu bulunamadı.")

df['processed_comment'] = df['comment'].astype(str).apply(preprocess_for_berturk)
print("Ön işleme tamamlandı.")


BERTurk için ön işleme (normalizasyonlu) BAŞLIYOR...
Ön işleme tamamlandı.


In [17]:
df.head()

,comment,film_name,processed_comment
0,Jean Reno denince zaten leon filmi gelir akla ...,Sevginin Gücü,jean reno denince zaten leon filmi gelir akla ...
1,Ekşın falan izlemek istiyorsanız eğer bunu izl...,Sevginin Gücü,ekşın falan izlemek istiyorsanız eğer bunu izl...
2,Bu yapım hakkında öyle çok şey yazabilirim ki ...,Sevginin Gücü,bu yapım hakkında oyle çok şey yazabilirim ki ...
3,finali yeter... (sting - shape of my heart)baz...,Sevginin Gücü,finali yeter sting shape of my heartbazıları f...
4,Jean Reno..bu adam kusursuz biri..ve oyunculug...,Sevginin Gücü,jean renobu adam kusursuz birive oyunculugu mu...


In [18]:
df_processed = df.drop(columns=["comment"])
df_processed.rename(columns={"processed_comment": "comment"}, inplace=True)

In [19]:
df_processed.head()

,film_name,comment
0,Sevginin Gücü,jean reno denince zaten leon filmi gelir akla ...
1,Sevginin Gücü,ekşın falan izlemek istiyorsanız eğer bunu izl...
2,Sevginin Gücü,bu yapım hakkında oyle çok şey yazabilirim ki ...
3,Sevginin Gücü,finali yeter sting shape of my heartbazıları f...
4,Sevginin Gücü,jean renobu adam kusursuz birive oyunculugu mu...


In [20]:
df_processed.to_csv("processed_turkish_movie_sentiment_dataset.csv", index=False)

In [21]:
from google.colab import files
files.download('processed_turkish_movie_sentiment_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>